In [1]:
import sys
import os
import math
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
# from osim.env import ProstheticsEnv
from prosthetics_env_with_history import ProstheticsEnvWithHistory
from td3 import TD3
# from replay_buffer import ReplayBuffer
from env_history_sampler import EnvHistorySampler


In [2]:
CONFIG = {
    "env": {
        "integrator_accuracy": 2e-3,
    },
    "model": {
        "architecture": "TD3",
    },
    "training": {
        "episode_save_load_file": "prosthetics_env_history.h5",
        "checkpoint_save_load_prefix": "checkpoint_TD3",
        "start_timesteps": 5e3, # How many time steps purely random policy is run for
        "eval_freq": 5e3, # How often (time steps) we evaluate
        "max_timesteps": 1e8, # Max time steps to train for
        "max_episode_steps": 300, # Max number of steps to run for a single episode
        "expl_noise": 0.5, # Std of Gaussian exploration noise  # was 0.1
        "batch_size": 100, # Batch size for both actor and critic
        "discount": 0.99, # Discount factor
        "tau": 0.005, # Target network update rate
        "policy_noise": 0.2, # Noise added to target policy during critic update
        "noise_clip": 0.5, # Range to clip target policy noise
        "policy_freq": 2, # Frequency of delayed policy updates
        "frameskip": 5, # Max Frameskip steps
    }
}


In [3]:
OUTPUT_DIR = Path('.')
LOGS_DIR = OUTPUT_DIR/'logs'
CHECKPOINTS_DIR = OUTPUT_DIR/'checkpoints'
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)
os.makedirs(CHECKPOINTS_DIR, exist_ok=True)


### Observation Hacking

- Rewrite all joint_pos, body_pos to be relative to mass_center_pos
- Subtract mass_center_vel and mass_center_acc from joint_vel, body_vel, joint_acc, body_acc?
- Either compute jounce/snap, or pass multiple timesteps, or just pass acceleration from past 3 timesteps?

Initial Env Observation:
```
{
    'joint_pos': {
        'ground_pelvis': [0.0, 0.0, 0.0, 0.0, 0.94, 0.0],
        'hip_r': [0.0, 0.0, 0.0],
        'knee_r': [0.0],
        'ankle_r': [0.0],
        'hip_l': [0.0, 0.0, 0.0],
        'knee_l': [0.0],
        'ankle_l': [0.0],
        'subtalar_l': [],
        'mtp_l': [],
        'back': [-0.0872665],
        'back_0': []
    },
    'joint_vel': {
        'ground_pelvis': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        'hip_r': [0.0, 0.0, 0.0],
        'knee_r': [0.0],
        'ankle_r': [0.0],
        'hip_l': [0.0, 0.0, 0.0],
        'knee_l': [0.0],
        'ankle_l': [0.0],
        'subtalar_l': [],
        'mtp_l': [],
        'back': [0.0],
        'back_0': []
    },
    'joint_acc': {
        'ground_pelvis': [34.07237489546962, 3.219284560937942, 0.021285761200362296, 13.997154494145377, 0.8655672359505977, -0.6156967622871027],
        'hip_r': [-194.74323476194263, -4.441803696780512, 1.5931700403370996e-14],
        'knee_r': [305.46152469620915],
        'ankle_r': [9636.363025843913],
        'hip_l': [-208.86020665024324, 3.5702556374966354, -4.2521541843143495e-14],
        'knee_l': [399.3192427973721],
        'ankle_l': [809.4478175113452],
        'subtalar_l': [],
        'mtp_l': [],
        'back': [-2.3092638912203256e-14],
        'back_0': []
    },
    'body_pos': {
        'pelvis': [0.0, 0.94, 0.0],
        'femur_r': [-0.0707, 0.8738999999999999, 0.0835],
        'pros_tibia_r': [-0.07519985651753601, 0.47807930355164957, 0.0835],
        'pros_foot_r': [-0.07519985651753601, 0.04807930355164958, 0.0835],
        'femur_l': [-0.0707, 0.8738999999999999, -0.0835],
        'tibia_l': [-0.07519985651753601, 0.47807930355164957, -0.0835],
        'talus_l': [-0.07519985651753601, 0.04807930355164958, -0.0835],
        'calcn_l': [-0.123969856517536, 0.006129303551649576, -0.09142],
        'toes_l': [0.05483014348246398, 0.004129303551649576, -0.0925],
        'torso': [-0.1007, 1.0214999999999999, 0.0],
        'head': [-0.052764320996907754, 1.5694070821576522, 0.0]
    },
    'body_vel': {
        'pelvis': [0.0, 0.0, 0.0],
        'femur_r': [0.0, 0.0, 0.0],
        'pros_tibia_r': [0.0, 0.0, 0.0],
        'pros_foot_r': [0.0, 0.0, 0.0],
        'femur_l': [0.0, 0.0, 0.0],
        'tibia_l': [0.0, 0.0, 0.0],
        'talus_l': [0.0, 0.0, 0.0],
        'calcn_l': [0.0, 0.0, 0.0],
        'toes_l': [0.0, 0.0, 0.0],
        'torso': [0.0, 0.0, 0.0],
        'head': [0.0, 0.0, 0.0]
    },
    'body_acc': {
        'pelvis': [13.997154494145377, 0.8655672359505977, -0.6156967622871027],
        'femur_r': [16.25111583579615, -1.812159929997423, -0.826986568448235],
        'pros_tibia_r': [-49.070641675940735, 0.12065763836075294, -0.34299240980632545],
        'pros_foot_r': [13.18934420084581, 0.12065763836075294, 0.18269081860597952],
        'femur_l': [16.24756111367569, -1.2745394083207864, -0.826986568448235],
        'tibia_l': [-55.19198970892064, 1.093538716356541, -0.6879691696202773],
        'talus_l': [41.356517039396714, 1.093538716356541, -0.537051606700039],
        'calcn_l': [84.73177709400595, -49.336407951145645, -0.5212902634646581],
        'toes_l': [86.79971256249173, 135.5386990655368, -0.5243942154141731],
        'torso': [11.220255940164602, -2.565520916023193, -0.35118159441778396],
        'head': [-7.448239570993795, -0.9322384901609437, 1.4116668685846663]
    },
    'body_pos_rot': {
        'pelvis': [-0.0, 0.0, -0.0],
        'femur_r': [-0.0, 0.0, -0.0],
        'pros_tibia_r': [-0.0, 0.0, -0.0],
        'pros_foot_r': [-0.0, 0.0, -0.0],
        'femur_l': [-0.0, 0.0, -0.0],
        'tibia_l': [-0.0, 0.0, -0.0],
        'talus_l': [-0.0, 0.0, -0.0],
        'calcn_l': [-0.0, 0.0, -0.0],
        'toes_l': [-0.0, 0.0, -0.0],
        'torso': [-0.0, 0.0, -0.0872665],
        'head': [-0.0, 0.0, -0.0872665]
    },
    'body_vel_rot': {
        'pelvis': [0.0, 0.0, 0.0],
        'femur_r': [0.0, 0.0, 0.0],
        'pros_tibia_r': [0.0, 0.0, 0.0],
        'pros_foot_r': [0.0, 0.0, 0.0],
        'femur_l': [0.0, 0.0, 0.0],
        'tibia_l': [0.0, 0.0, 0.0],
        'talus_l': [0.0, 0.0, 0.0],
        'calcn_l': [0.0, 0.0, 0.0],
        'toes_l': [0.0, 0.0, 0.0],
        'torso': [0.0, 0.0, 0.0],
        'head': [0.0, 0.0, 0.0]
    },
    'body_acc_rot': {
        'pelvis': [3.219284560937942, 0.021285761200362296, 34.07237489546962],
        'femur_r': [-1.2225191358425698, 0.021285761200378228, -160.670859866473],
        'pros_tibia_r': [-1.2225191358425698, 0.021285761200378228, 144.79066482973616],
        'pros_foot_r': [-1.2225191358425698, 0.021285761200378228, 9781.15369067365],
        'femur_l': [-0.35097107655869353, 0.021285761200404818, -174.7878317547736],
        'tibia_l': [-0.35097107655869353, 0.021285761200404818, 224.5314110425985],
        'talus_l': [-0.35097107655869353, 0.021285761200404818, 1033.9792285539438],
        'calcn_l': [-0.35097107655869353, 0.021285761200404818, 1033.9792285539438],
        'toes_l': [-0.35097107655869353, 0.021285761200404818, 1033.9792285539438],
        'torso': [3.219284560937942, 0.021285761200362296, 34.0723748954696],
        'head': [3.219284560937942, 0.021285761200362296, 34.0723748954696]
    },
    'forces': {
        'abd_r': [219.6613927253564],
        'add_r': [144.87433100305103],
        'hamstrings_r': [194.30030504346755],
        'bifemsh_r': [42.728811234363775],
        'glut_max_r': [171.7873509605573],
        'iliopsoas_r': [158.01207984383657],
        'rect_fem_r': [99.0329705435046],
        'vasti_r': [436.79388413623326],
        'abd_l': [219.6613927253564],
        'add_l': [144.87433100305103],
        'hamstrings_l': [194.30030504346755],
        'bifemsh_l': [42.728811234363775],
        'glut_max_l': [171.7873509605573],
        'iliopsoas_l': [158.01207984383657],
        'rect_fem_l': [99.0329705435046],
        'vasti_l': [436.79388413623326],
        'gastroc_l': [273.0178325689043],
        'soleus_l': [370.0059951709156],
        'tib_ant_l': [104.05059952034294],
        'ankleSpring': [-0.0],
        'pros_foot_r_0': [-1.3573320551122304e-12, -388.7553514927188, 0.0, 32.46107184964201, -1.1333722660187127e-13, 3.726164264482634, 1.3573320551122304e-12, 388.7553514927188, 0.0, 0.0, 0.0, 25.50818238819416, 1.3573320551122304e-12, 388.7553514927188, 0.0, 0.0, 0.0, 25.50818238819416],
        'foot_l': [-1.7615592933859115e-12, -504.53063397142085, 0.0, -46.45915575255036, 1.622112753284362e-13, -4.943148065393853, 6.786660275561278e-13, 194.37767574636297, 0.0, 0.0, 0.0, 5.831330272390875, 1.0828932658297836e-12, 310.1529582250579, 0.0, 0.0, 0.0, 6.203059164501164, 1.0828932658297836e-12, 310.1529582250579, 0.0, 0.0, 0.0, 6.203059164501164],
        'HipLimit_r': [0.0, 0.0],
        'HipLimit_l': [0.0, 0.0],
        'KneeLimit_r': [-0.0, 0.0],
        'KneeLimit_l': [-0.0, 0.0],
        'AnkleLimit_r': [0.0, 0.0],
        'AnkleLimit_l': [0.0, 0.0],
        'HipAddLimit_r': [0.0, 0.0],
        'HipAddLimit_l': [0.0, 0.0]
    },
    'muscles': {
        'abd_r': {
            'activation': 0.05,
            'fiber_length': 0.07752306863700548,
            'fiber_velocity': 1.1700156898117815e-13,
            'fiber_force': 219.6613927253564
        },
        'add_r': {
            'activation': 0.05,
            'fiber_length': 0.05526137592854144,
            'fiber_velocity': 5.531257930905764e-11,
            'fiber_force': 146.25768888087705
        },
        'hamstrings_r': {
            'activation': 0.05,
            'fiber_length': 0.06355896214015513,
            'fiber_velocity': 2.1056261406660054e-14,
            'fiber_force': 202.45627069225887
        },
        'bifemsh_r': {
            'activation': 0.05,
            'fiber_length': 0.13434264681417835,
            'fiber_velocity': 9.542198984660805e-17,
            'fiber_force': 45.09919197278584
        },
        'glut_max_r': {
            'activation': 0.05,
            'fiber_length': 0.16084824667171801,
            'fiber_velocity': 1.0181982508865008e-12,
            'fiber_force': 171.7873509605573
        },
        'iliopsoas_r': {
            'activation': 0.05,
            'fiber_length': 0.13005768603600326,
            'fiber_velocity': 3.347183497651294e-11,
            'fiber_force': 159.26525950285387
        },
        'rect_fem_r': {
            'activation': 0.05,
            'fiber_length': 0.06027044615978444,
            'fiber_velocity': 2.2362024955832438e-15,
            'fiber_force': 99.63652479982161
        },
        'vasti_r': {
            'activation': 0.05,
            'fiber_length': 0.07890756873654925,
            'fiber_velocity': 6.168233828156989e-15,
            'fiber_force': 437.7385693769557
        },
        'abd_l': {
            'activation': 0.05,
            'fiber_length': 0.07752306863700548,
            'fiber_velocity': 1.1700156898117815e-13,
            'fiber_force': 219.6613927253564
        },
        'add_l': {
            'activation': 0.05,
            'fiber_length': 0.05526137592854144,
            'fiber_velocity': 5.531257930905764e-11,
            'fiber_force': 146.25768888087705
        },
        'hamstrings_l': {
            'activation': 0.05,
            'fiber_length': 0.06355896214015513,
            'fiber_velocity': 2.1056261406660054e-14,
            'fiber_force': 202.45627069225887
        },
        'bifemsh_l': {
            'activation': 0.05,
            'fiber_length': 0.13434264681417835,
            'fiber_velocity': 9.542198984660805e-17,
            'fiber_force': 45.09919197278584
        },
        'glut_max_l': {
            'activation': 0.05,
            'fiber_length': 0.16084824667171801,
            'fiber_velocity': 1.0181982508865008e-12,
            'fiber_force': 171.7873509605573
        },
        'iliopsoas_l': {
            'activation': 0.05,
            'fiber_length': 0.13005768603600326,
            'fiber_velocity': 3.347183497651294e-11,
            'fiber_force': 159.26525950285387
        },
        'rect_fem_l': {
            'activation': 0.05,
            'fiber_length': 0.06027044615978444,
            'fiber_velocity': 2.2362024955832438e-15,
            'fiber_force': 99.63652479982161
        },
        'vasti_l': {
            'activation': 0.05,
            'fiber_length': 0.07890756873654925,
            'fiber_velocity': 6.168233828156989e-15,
            'fiber_force': 437.7385693769557
        },
        'gastroc_l': {
            'activation': 0.05,
            'fiber_length': 0.05720257668702345,
            'fiber_velocity': 5.718949639274886e-14,
            'fiber_force': 282.79456495087237
        },
        'soleus_l': {
            'activation': 0.05,
            'fiber_length': 0.04494814124106819,
            'fiber_velocity': 3.4120643802478774e-10,
            'fiber_force': 406.4161372187392
        },
        'tib_ant_l': {
            'activation': 0.05,
            'fiber_length': 0.06288000983990409,
            'fiber_velocity': 8.525642140971546e-14,
            'fiber_force': 104.5158690359632
        }
    },
    'markers': {},
    'misc': {
        'mass_center_pos': [-0.08466565561225976, 0.9952730567231536, -0.003576087446004414],
        'mass_center_vel': [0.0, 0.0, 0.0],
        'mass_center_acc': [4.4008799039045146e-14, 2.570832209970726, -1.0237645330147003e-15]
    }
}
```

In [4]:
# Copied & modified from https://github.com/stanfordnmbl/osim-rl/blob/master/osim/env/osim.py#L452
def single_step_env_obs_to_model_obs(env_obs, target_vel=[3,0,0], include_lower_order_values=True):
    env_obs = env_obs.copy()
    has_prosthetic = 'pros_foot_r' in env_obs['body_pos']

    target_vel_x = target_vel[0]
    target_vel_z = target_vel[2]
    eps = 1e-8

    frame = {
        'pos': np.array(env_obs['misc']['mass_center_pos']),
        'vel': np.array(env_obs['misc']['mass_center_vel']),
        'acc': np.array(env_obs['misc']['mass_center_acc']),
    }

    # Transform reference frame from 0,0,0 to center of mass:
    for k, pos in env_obs['body_pos'].items():
        env_obs['body_pos'][k] = list(np.array(pos) - frame['pos'])
    for k, vel in env_obs['body_vel'].items():
        env_obs['body_vel'][k] = list(np.array(vel) - frame['vel'])
    for k, acc in env_obs['body_acc'].items():
        env_obs['body_acc'][k] = list(np.array(acc) - frame['acc'])

    # Normalize body vel/acc based on center of mass vel/acc:
#     for k, vel in env_obs['body_vel'].items():
#         env_obs['body_vel'][k] = list(np.array(vel) / (frame['vel'] + eps))
#     for k, acc in env_obs['body_acc'].items():
#         env_obs['body_acc'][k] = list(np.array(acc) / (frame['acc'] + eps))

    # Collect observation vector
    lower_order = []
    highest_order = []    
    for body_part in ["head","torso","pelvis","femur_l","femur_r","tibia_l","tibia_r","pros_tibia_r","talus_l","talus_r","toes_l","toes_r","pros_foot_r","calcn_l","calcn_r"]:
        if has_prosthetic and body_part in ["toes_r","tibia_r","talus_r","calcn_r"]:
            lower_order += [0] * 12
            highest_order += [0] * 6
            continue
        if not has_prosthetic and body_part in ["pros_foot_r","pros_tibia_r"]:
            lower_order += [0] * 12
            highest_order += [0] * 6
            continue
        lower_order += env_obs["body_pos"][body_part][0:2]
        lower_order += env_obs["body_vel"][body_part][0:2]
        highest_order += env_obs["body_acc"][body_part][0:2]
        lower_order += env_obs["body_pos_rot"][body_part][0:2]
        lower_order += env_obs["body_vel_rot"][body_part][0:2]
        highest_order += env_obs["body_acc_rot"][body_part][0:2]

    for joint in ["ground_pelvis","ankle_l","ankle_r","back","hip_l","hip_r","knee_l","knee_r"]:
        lower_order += env_obs["joint_pos"][joint]
        lower_order += env_obs["joint_vel"][joint]
        highest_order += env_obs["joint_acc"][joint]

    for muscle in sorted(env_obs["muscles"].keys()):
        highest_order += [env_obs["muscles"][muscle]["activation"]]
        highest_order += [env_obs["muscles"][muscle]["fiber_length"]]
        highest_order += [env_obs["muscles"][muscle]["fiber_velocity"]]
        highest_order += [env_obs["muscles"][muscle]["fiber_force"]]

    for force in ['abd_l', 'add_l', 'hamstrings_l', 'bifemsh_l', 'glut_max_l', 'iliopsoas_l', 'rect_fem_l', 'vasti_l', 'gastroc_l', 'soleus_l', 'tib_ant_l', 'ankleSpring', 'pros_foot_r_0', 'foot_l', 'HipLimit_l', 'KneeLimit_l', 'AnkleLimit_l', 'HipAddLimit_l']:
        highest_order += env_obs['forces'][force]
        if not '_l' in force:
            continue
        force = force.replace('_l', '_r')
        if has_prosthetic:
            if force in ['gastroc_r', 'soleus_r', 'tib_ant_r']:
                highest_order += [0]
                continue
            if force in ['foot_r']:
                highest_order += [0] * 24
                continue
        else:
            if force in ['pros_foot_r_0']:
                highest_order += [0] * 18
                continue
        highest_order += env_obs['forces'][force.replace('_l', '_r')]

    # Center of mass
    lower_order += list(frame['pos'])
    lower_order += list(frame['vel'])
    highest_order += list(frame['acc'])

    # Target velocity
    highest_order += [frame['vel'][0] - target_vel_x, frame['vel'][2] - target_vel_z]

    result = highest_order
    if include_lower_order_values:
        result = lower_order + result
    return result

def env_obs_history_to_model_obs(env_obs_history):
    env_obs_history = env_obs_history[-4:]
    # Duplicate first env_obs to ensure we have at least 4 steps of history.
    env_obs_history = env_obs_history[:1] * (4 - len(env_obs_history)) + env_obs_history

    model_obs_steps = [single_step_env_obs_to_model_obs(env_obs_history[-1])]
    model_obs_steps += [single_step_env_obs_to_model_obs(env_obs, include_lower_order_values=False) for env_obs in env_obs_history[:-1]][::-1]
    return np.concatenate(model_obs_steps)


def prepare_model_observation(env):
    df_history = env.history(current_episode_only=True)
    model_obs = env_obs_history_to_model_obs(df_history['obs'].tolist())
    return model_obs


### Action hacking

- binary activations?  Or several "bins"?
- Frameskip?
- Muscles must remain "active" for at least 10 frames once activated?  Randomized?
- Limited number of muscles can fire at one time?
- Handle prosthetic or not (strip activations of nonexistent muscles)


In [5]:
action_state = {}

def reset_frameskip(n=8):
    action_state['n_frameskip'] = n
    action_state['frames_to_skip'] = 0  # Start at 0 so we don't skip the first model action.
    action_state['frameskip_action'] = None
    
def apply_frameskip(model_action):
    if action_state.get('frames_to_skip', 0) == 0:
        # Already skipped enough frames.  Reset counter & cache unskipped frame.
        action_state['frames_to_skip'] = np.random.randint(action_state.get('n_frameskip', 0) + 1)
        action_state['frameskip_action'] = model_action
    else:
        # Need to skip this frame.  Decrement the counter & apply the cached unskipped frame.
        action_state['frames_to_skip'] -= 1
        model_action = action_state.get('frameskip_action')
    return model_action


def prepare_env_action(model_action):
    # model_action is a list of muscle activations

    # Frame skipping
    model_action = apply_frameskip(model_action)

    model_action = np.array(model_action)

    # Binarize the muscle activations
    model_action = model_action.round()
    
    return model_action.tolist()
    

### Reward Hacking

- Survival reward
- Lean forward reward (to avod models which had torso too upright) (may need to be based on speed)
- Reward for minimizing sideways velocity?
- Reward for minimizing vertical velocity (COG)?

In [6]:
def env_obs_to_custom_reward(obs):
    if type(obs) != dict:
        raise ValueError('obs must be a dict (project=False)')

    target_vel_x = 3
    target_vel_z = 0
    eps = 1e-8

    target_vel_theta = -np.arctan(target_vel_z/(target_vel_x+eps)) if target_vel_x >= 0 else np.pi - np.arctan(target_vel_z/(target_vel_x+eps))

    # Parabolic reward/penalty for tracking a target value within some tolerance. 
    # Unit reward: 1 at target value, 0 at `tolerance` distance from target value, negative outside of `tolerance`.
    # Multiply by desired scale factor / magnitude. Don't multiply by too large of a scale factor — it also amplifies the slope.
    # Make the tolerance bigger than you think — it makes the slope more gradual / less severe.
    # `tolerance`: reward is positive if within `tolerance` of target value, else negative.
    val_diff_reward = lambda val, target, tolerance: (1 - ((val - target)/tolerance)**2)
    def radians_diff_wrapped(a1, a2):
        # Make both angles within (-2pi, 2pi)
        a1, a2 = a1 % (2*np.pi), a2 % (2*np.pi)
        # Make both angles positive -- within [0, 2pi)
        a1 = a1 + 2*np.pi if a1 < 0 else a1
        a2 = a2 + 2*np.pi if a2 < 0 else a2
        # Make a1 the smaller angle
        a1, a2 = min(a1, a2), max(a1, a2)
        # Make sure a1 is within pi of a2 — two angles can't be greater than pi apart in relative (wrapped) sense.
        a1 = a1 + 2*np.pi if a2 - a1 > np.pi else a1
        return np.fabs(a2 - a1)
    
    avg_knee_joint_pos = (obs['joint_pos']['knee_l'][0] + obs['joint_pos']['knee_r'][0]) / 2
    target_avg_knee_joint_pos = 60*np.pi/180
    
    rewards = {
        'survival': 100,
        'target_velocity_x': 3 * val_diff_reward(obs['misc']['mass_center_vel'][0], target_vel_x, 5), # 3 at target velocity, 0 at 3m/s off-target, then negative
        'target_velocity_z': 3 * val_diff_reward(obs['misc']['mass_center_vel'][2], target_vel_z, 5), # 3 at target velocity, 0 at 3m/s off-target, then negative
        'head_velocity_x': 2 * val_diff_reward(obs['body_vel']['head'][0], target_vel_x, 5), # 2 at target velocity, 0 at 3m/s off-target, then negative
        'head_velocity_z': 2 * val_diff_reward(obs['body_vel']['head'][2], target_vel_z, 5), # 2 at target velocity, 0 at 3m/s off-target, then negative
        'lean_forward_x': 5 * val_diff_reward(obs['body_pos']['head'][0] - obs['body_pos']['pelvis'][0], .1 * target_vel_x, .4), # head in front of pelvis from perspective of velocity vector
        'lean_forward_z': 5 * val_diff_reward(obs['body_pos']['head'][2] - obs['body_pos']['pelvis'][2], .1 * target_vel_z, .4), # head in front of pelvis from perspective of velocity vector
        'hips_squared': 5 * val_diff_reward(radians_diff_wrapped(target_vel_theta, obs['body_pos_rot']['pelvis'][1]), 0, np.pi),
        'knee_bent_l': 5 * val_diff_reward(obs['joint_pos']['knee_l'][0], target_avg_knee_joint_pos, np.pi), # goal range of roughly [0,120] degrees
        'knee_bent_r': 5 * val_diff_reward(obs['joint_pos']['knee_r'][0], target_avg_knee_joint_pos, np.pi), # goal range of roughly [0,120] degrees
        'low_y_vel_pelvis': 5 * val_diff_reward(obs['body_vel']['pelvis'][1], 0, 1),
        'low_y_vel_head': 5 * val_diff_reward(obs['body_vel']['head'][1], 0, 1),
        'low_y_vel_toes_l': 5 * val_diff_reward(obs['body_vel']['toes_l'][1], 0, 1),
        'low_y_vel_pros_foot_r': 5 * val_diff_reward(obs['body_vel']['pros_foot_r'][1], 0, 1),
        'knees_opposite_joint_vel': 0 if avg_knee_joint_pos < (target_avg_knee_joint_pos - 15*np.pi/180) else 3 * val_diff_reward(obs['joint_vel']['knee_l'][0], -obs['joint_vel']['knee_r'][0], np.pi), # The left knee should be opening when the right knee is closing, and vice versa
        'feet_behind_mass_x': 5 * val_diff_reward(obs['misc']['mass_center_pos'][0] - (obs['body_pos']['toes_l'][0] + obs['body_pos']['pros_foot_r'][0])/2, .1 * target_vel_x, .4),
        'feet_behind_mass_z': 5 * val_diff_reward(obs['misc']['mass_center_pos'][2] - (obs['body_pos']['toes_l'][2] + obs['body_pos']['pros_foot_r'][2])/2, .1 * target_vel_z, .4),
        'one_foot_off_ground': 0,
        'femurs_parallel': 0,
        'absolute_foot_velocity': 0, # should be 0 at ground, 2x body velocity above ground
        'forefoot_strike': 0,
    }    

#     if should_abort_episode(obs, custom_rewards=rewards):
#         rewards['abort_episode'] = -100

    return rewards
    

### Episode Hacking (Custom "done" criteria)


In [7]:
def should_abort_episode(env_obs, custom_rewards=None, verbose=False):
#     print((np.array(env_obs['body_pos_rot']['torso'])*180/math.pi > 60).any())
#     if env_obs['body_pos_rot']['torso'][2] < -0.2:
#         return True
    rewards = custom_rewards if custom_rewards != None else env_obs_to_custom_reward(env_obs)
    # print(f'Custom reward: {sum(rewards.values())}')
    if (env_obs['body_pos']['head'][0] - env_obs['body_pos']['pelvis'][0]) < -.2:
        if verbose: print(f'Aborting episode due to head being > .2m behind the pelvis ({env_obs["body_pos"]["head"][0] - env_obs["body_pos"]["pelvis"][0]}).')
        return True
    if np.fabs(env_obs['body_pos']['head'][2]) > 0.5:
        if verbose: print(f'Aborting episode due to head being > 0.5m away from centerline ({env_obs["body_pos"]["head"][2]}).')
        return True
    if sum(rewards.values()) < -10:
        if verbose:
            print(f'Aborting episode due to custom reward < -10 ({sum(rewards.values())}):')
            for k,v in rewards.items():
                if v < 0:
                    print(f'  reward `{k}` = {v}')
        return True
    return False
    

### Policy Evaluator

In [8]:
# Runs policy for X episodes and returns average reward
def evaluate_episode(policy):
    obs = env.reset(**env_step_kwargs)
    reset_frameskip(0)
    done = False
    total_reward = 0
    while not done:
        action = policy.select_action(prepare_model_observation(env))
        action = prepare_env_action(action)
        obs, reward, done, _ = env.step(action, **env_step_kwargs)
        
        # We don't use the custom rewards here because we want to evaluate our progress against the environment's reward.
        # obs_dict = env.get_state_desc()
        # custom_rewards = compute_rewards(obs_dict)
        # total_rewared += reward + sum(custom_rewards.values())

        total_reward += reward
    return total_reward

def evaluate_policy(policy, eval_episodes=10):
    avg_reward = 0.
    for _ in tqdm(range(eval_episodes), desc="Evaluating policy", unit="episode"):
        avg_reward += evaluate_episode(policy)

    avg_reward /= eval_episodes

    print("---------------------------------------")
    print("Evaluation over %d episodes: %f" % (eval_episodes, avg_reward))
    print("---------------------------------------")
    return avg_reward



In [9]:
env = ProstheticsEnvWithHistory(visualize=False, integrator_accuracy=CONFIG['env']['integrator_accuracy'])
env_step_kwargs = {'project': False}

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [10]:
# state_dim = env.observation_space.shape[0]
env.reset(**env_step_kwargs)
state_dim = prepare_model_observation(env).shape[0]
action_dim = env.action_space.shape[0]
max_action = int(env.action_space.high[0])
state_dim, action_dim, max_action


(1260, 19, 1)

In [11]:
policy = TD3(state_dim, action_dim, max_action)

In [12]:
try:
    print(f'Loading checkpoint from {CHECKPOINTS_DIR/CONFIG["training"]["checkpoint_save_load_prefix"]}*')
    policy.load(CHECKPOINTS_DIR, CONFIG["training"]["checkpoint_save_load_prefix"])
except:
    print("Failed to load existing model checkpoint — this is normal if you're trying to train a new model (not from a checkpoint).")
    pass


Loading checkpoint from checkpoints/checkpoint_TD3*
Failed to load existing model checkpoint — this is normal if you're trying to train a new model (not from a checkpoint).


In [13]:
# replay_buffer = ReplayBuffer()

In [14]:
# %%timeit -n1 -r1
# evaluations = [evaluate_policy(policy)]
evaluations = []


In [15]:
# Load saved episodes from disk
if os.path.exists(CONFIG['training']['episode_save_load_file']):
    df_saved_episodes = pd.read_hdf(CONFIG['training']['episode_save_load_file'])
else:
    df_saved_episodes = pd.DataFrame(columns=env.history().columns)
    
print(f'Loaded {len(df_saved_episodes)} saved episode timesteps.')


Loaded 0 saved episode timesteps.


In [16]:
# Initialize based on loaded episode history
total_timesteps = len(df_saved_episodes)
timesteps_since_eval = len(df_saved_episodes) % CONFIG['training']['eval_freq']
episode_num = len(df_saved_episodes['episode_uuid'].unique()) - 1
done = True
episode_timesteps = df_saved_episodes['i_step'].iloc[-1] if total_timesteps > 0 else 0
total_timesteps, timesteps_since_eval, episode_num, episode_timesteps


(0, 0.0, -1, 0)

In [17]:
while total_timesteps < CONFIG['training']['max_timesteps']:
    if done: 
        if total_timesteps >= CONFIG['training']['start_timesteps']: 
            df_env_history = pd.concat([df_saved_episodes, env.history()], ignore_index=True, copy=False)
            history_sampler = EnvHistorySampler(
                df_env_history,
                env_obs_history_to_model_obs_fn=env_obs_history_to_model_obs, 
                env_obs_custom_reward_fn=lambda obs: sum(env_obs_to_custom_reward(obs).values()),
                env_obs_custom_done_fn=should_abort_episode,
            )
            if CONFIG['model']['architecture'] == "TD3":
                policy.train(
                    history_sampler,#replay_buffer, 
                    episode_timesteps, 
                    CONFIG['training']['batch_size'], 
                    CONFIG['training']['discount'], 
                    CONFIG['training']['tau'], 
                    CONFIG['training']['policy_noise'], 
                    CONFIG['training']['noise_clip'], 
                    CONFIG['training']['policy_freq'],
                )
            else: 
                policy.train(
                    history_sampler,#replay_buffer, 
                    episode_timesteps, 
                    CONFIG['training']['batch_size'], 
                    CONFIG['training']['discount'], 
                    CONFIG['training']['tau']
                )
        
            # Evaluate policy, Checkpoint policy, Checkpoint history
            if timesteps_since_eval >= CONFIG['training']['eval_freq']:
                # Reset evaluation counter
                timesteps_since_eval %= CONFIG['training']['eval_freq']
                # Evaluate policy
                evaluations.append(evaluate_policy(policy))
                # Checkpoint policy
                policy.save(CHECKPOINTS_DIR, f'{CONFIG["training"]["checkpoint_save_load_prefix"]}_episode{episode_num}_eval{evaluations[-1]:.1f}')
                policy.save(CHECKPOINTS_DIR, CONFIG["training"]["checkpoint_save_load_prefix"])
                # Checkpoint history
                pd.concat([df_saved_episodes, env.history()], ignore_index=True, copy=False).to_hdf(CONFIG['training']['episode_save_load_file'], key='df')
                # TODO Log evaluations, etc.
        
        # Reset environment
        obs = env.reset(**env_step_kwargs)
        reset_frameskip(CONFIG['training']['frameskip'])
#         obs_dict = env.get_state_desc()
        done = False
        episode_reward = 0
        episode_timesteps = 0
        episode_num += 1 
    
    # Select action randomly or according to policy
    if total_timesteps < CONFIG['training']['start_timesteps']:
        action = env.action_space.sample()
    else:
        action = policy.select_action(prepare_model_observation(env))
        if CONFIG['training']['expl_noise'] != 0: 
            action += np.random.normal(0, CONFIG['training']['expl_noise'], size=action.shape)

    # Perform action
    action = prepare_env_action(action)
    obs, reward, done, _ = env.step(action, **env_step_kwargs)
#     new_obs_dict = env.get_state_desc()

    if not done:
        done = should_abort_episode(env.get_state_desc(), verbose=True)
    done_bool = 0 if episode_timesteps + 1 == CONFIG['training']['max_episode_steps'] else float(done)

    # custom_rewards = compute_rewards(new_obs_dict)
    episode_reward += reward #+ sum(custom_rewards.values())

    # Store data in replay buffer
#     replay_buffer.add((obs_dict, new_obs_dict, action, reward, done_bool, episode_num))

#     obs = new_obs
#     obs_dict = new_obs_dict

    episode_timesteps += 1
    total_timesteps += 1
    timesteps_since_eval += 1
    
    if done:
        print(f"Total T: {total_timesteps} Episode Num: {episode_num} Episode T: {episode_timesteps} Reward: {episode_reward}")
        sys.stdout.flush()

        

-0.13876400884778411 0.06273868911010606
Aborting episode due to head being > .2m behind the pelvis (-0.20150269795789016).
Total T: 11 Episode Num: 0 Episode T: 11 Reward: 35.43506634798316
-0.15462359669252684 0.059591450039721064
Aborting episode due to head being > .2m behind the pelvis (-0.2142150467322479).
Total T: 27 Episode Num: 1 Episode T: 16 Reward: 34.57707824491329
-0.1503674592430194 0.055268684997500296
Aborting episode due to head being > .2m behind the pelvis (-0.20563614424051968).
Total T: 47 Episode Num: 2 Episode T: 20 Reward: 31.886474990890747
-0.13486544085689717 0.06552271343739556
Aborting episode due to head being > .2m behind the pelvis (-0.20038815429429274).
Total T: 57 Episode Num: 3 Episode T: 10 Reward: 35.54114907912499
-0.14341715950241202 0.07355862041109143
Aborting episode due to head being > .2m behind the pelvis (-0.21697577991350345).
Total T: 69 Episode Num: 4 Episode T: 12 Reward: 41.12720894055058
-0.14645787204372127 0.062358306584047694
Ab

-0.19147309075965144 0.019800976817729323
Aborting episode due to head being > .2m behind the pelvis (-0.21127406757738076).
Total T: 711 Episode Num: 43 Episode T: 27 Reward: 11.335725951884346
-0.14019375239456092 0.06262545976276687
Aborting episode due to head being > .2m behind the pelvis (-0.2028192121573278).
Total T: 724 Episode Num: 44 Episode T: 13 Reward: 35.24279424516603
-0.14415453227333672 0.07207441002376652
Aborting episode due to head being > .2m behind the pelvis (-0.21622894229710324).
Total T: 739 Episode Num: 45 Episode T: 15 Reward: 40.65239371405581
-0.1448257042497914 0.0646612081391531
Aborting episode due to head being > .2m behind the pelvis (-0.2094869123889445).
Total T: 752 Episode Num: 46 Episode T: 13 Reward: 36.95092482782803
-0.15432121187606407 0.055598748394860294
Aborting episode due to head being > .2m behind the pelvis (-0.20991996027092436).
Total T: 767 Episode Num: 47 Episode T: 15 Reward: 32.62538085996184
-0.15128978390600872 0.0586599140336

-0.13931820258087088 0.07217063066935563
Aborting episode due to head being > .2m behind the pelvis (-0.2114888332502265).
Total T: 1339 Episode Num: 86 Episode T: 12 Reward: 40.553722333557694
-0.1399440040539145 0.06885819220436878
Aborting episode due to head being > .2m behind the pelvis (-0.20880219625828328).
Total T: 1350 Episode Num: 87 Episode T: 11 Reward: 38.10137982614998
-0.14594495065143387 0.06896484878896438
Aborting episode due to head being > .2m behind the pelvis (-0.21490979944039823).
Total T: 1361 Episode Num: 88 Episode T: 11 Reward: 38.480205618541525
-0.1364743646336566 0.06412807718833341
Aborting episode due to head being > .2m behind the pelvis (-0.20060244182199).
Total T: 1376 Episode Num: 89 Episode T: 15 Reward: 37.04592650260125
-0.3603978993217228 -0.15803864313633492
Aborting episode due to head being > .2m behind the pelvis (-0.20235925618538786).
Total T: 1426 Episode Num: 90 Episode T: 50 Reward: -105.03291642163438
-0.15193153622342423 0.070183338

-0.1437607126500197 0.06282431160971982
Aborting episode due to head being > .2m behind the pelvis (-0.2065850242597395).
Total T: 2017 Episode Num: 129 Episode T: 19 Reward: 36.56007384574263
-0.1436047925697776 0.06490939176405872
Aborting episode due to head being > .2m behind the pelvis (-0.2085141843338363).
Total T: 2030 Episode Num: 130 Episode T: 13 Reward: 36.97525512259188
-0.14801218627841506 0.06867130991908822
Aborting episode due to head being > .2m behind the pelvis (-0.21668349619750327).
Total T: 2042 Episode Num: 131 Episode T: 12 Reward: 38.098395295080394
-0.14485072846601943 0.06678800140191749
Aborting episode due to head being > .2m behind the pelvis (-0.21163872986793691).
Total T: 2054 Episode Num: 132 Episode T: 12 Reward: 37.43504005044569
-0.14141464401918102 0.06378561057443732
Aborting episode due to head being > .2m behind the pelvis (-0.20520025459361835).
Total T: 2067 Episode Num: 133 Episode T: 13 Reward: 36.362276331703356
-0.1467449434211912 0.06030

-0.14773710897261974 0.058067418161572834
Aborting episode due to head being > .2m behind the pelvis (-0.20580452713419256).
Total T: 2681 Episode Num: 172 Episode T: 14 Reward: 33.20906181432305
-0.1714146298671788 0.039032673165753345
Aborting episode due to head being > .2m behind the pelvis (-0.21044730303293216).
Total T: 2704 Episode Num: 173 Episode T: 23 Reward: 23.140109676331498
-0.14474666311548878 0.06967614074030215
Aborting episode due to head being > .2m behind the pelvis (-0.21442280385579093).
Total T: 2716 Episode Num: 174 Episode T: 12 Reward: 39.403282680947214
-0.1536002113615742 0.04887021858197665
Aborting episode due to head being > .2m behind the pelvis (-0.20247042994355086).
Total T: 2733 Episode Num: 175 Episode T: 17 Reward: 28.398601796289583
-0.14803952769258233 0.06614584986258834
Aborting episode due to head being > .2m behind the pelvis (-0.21418537755517067).
Total T: 2747 Episode Num: 176 Episode T: 14 Reward: 37.38121069572466
-0.14168753419589664 0

...

Train model: 100%|██████████| 78/78 [01:36<00:00,  1.20s/batch]


Aborting episode due to head being > 0.5m away from centerline (-0.5172219346787169).
Total T: 25111 Episode Num: 532 Episode T: 176 Reward: -90.95824123347685


Evaluating policy: 100%|██████████| 10/10 [03:41<00:00, 22.12s/episode]


---------------------------------------
Evaluation over 10 episodes: -80.558020
---------------------------------------


MemoryError: 

In [ ]:
!sudo shutdown

In [65]:
# pd.concat([df_saved_episodes, env.history()], ignore_index=True, copy=False)
# df_saved_episodes.to_hdf(CONFIG['training']['episode_save_load_file'].replace('.','_start.'), key='df', format='table')
# env.history().to_hdf(CONFIG['training']['episode_save_load_file'].replace('.','_start.'), key='df', format='table', append=True)
pd.concat([df_saved_episodes, env_history], ignore_index=True, copy=False).to_hdf(CONFIG['training']['episode_save_load_file'], key='df')
# env.history()['i_step'].dtype
# df_saved_episodes['i_step'].dtype


/home/ubuntu/.local/share/virtualenvs/nips-2018-ai-for-prosthetics-fCqIkKV7/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['episode_uuid', 'i_step', 'action', 'obs', 'reward', 'done', 'info']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


MemoryError: 